# Reconhecimento de imagens: Twitter e Computer Vision API

## Bibliotecas básicas e outras inicializações

In [1]:
import tweepy as tw
%run credentials/credentials.py

## APP Twitter

### Autenticação

In [2]:
api_key = credentials['api_key']
api_secret_key = credentials['api_secret_key']
access_token = credentials['access_token']
access_token_secret = credentials['access_token_secret']

### Primeiro tweet

In [3]:
auth = tw.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)
tweet = api.update_status("Python e Twitter na #Alura")

### Pesquisando tweets

In [4]:
tweets = tw.Cursor(api.search, q='data science', since='2021-01-01', lang='pt').items(5)
for tweet in tweets:
    print(tweet.user.screen_name)
    print(tweet.user.location)
    print(tweet.text)
    print('-----------------------------------------------------------------------------\n')

paanalytics1
São Paulo
2021-05-08 11:00:05.510105 - Consultoria em Data Science e Analytics: Análise de Dados, Machine Learning, Construçã… https://t.co/stXkIyDQcv
-----------------------------------------------------------------------------

Luis_OrEa
Recife, Brasil
RT @curso_r: @anarina @gilnogueiraofc Bora vigoRaR estudando R com a gente, @gilnogueiraofc ? Inclusive acabamos de liberar o cupom GILDOVI…
-----------------------------------------------------------------------------

daft_zebra_
Now
Data science é cornojob gourmet e eu posso provar
-----------------------------------------------------------------------------

hnrqlds
Brazil
Agora esses picaretas inveterados da Empiricus se deram conta que data science é buzzword e FLOODARAM o youtube com ads sobre isso.
-----------------------------------------------------------------------------

SoyRafa28309693

@anarina @gilnogueiraofc @curso_r Tem algum programa da Globo pra jovens na área de Data Science?
--------------------------

### Pesquisando famosos

In [5]:
famosos = ['cauareymond', 'ivetesangalo', 'Tatawerneck']

for famoso in famosos:
    tweets = tw.Cursor(api.search, q=famoso, since='2019-01-01').items(20)
    print('Autor do tweet: ', famoso)
    print('Imagens postadas: ')
    for tweet in tweets:
        if 'media' in tweet.entities:
            print(tweet.entities['media'][0]['media_url'])
            
    print('---------------------------\n')

Autor do tweet:  cauareymond
Imagens postadas: 
http://pbs.twimg.com/media/E0y2nfOWUAgU6n2.jpg
http://pbs.twimg.com/media/Ezw2S6hWQAIRzCy.jpg
http://pbs.twimg.com/media/Ez77yR9WQAAnjuD.jpg
http://pbs.twimg.com/ext_tw_video_thumb/1390747675021168645/pu/img/X74BSpzO0Nnb7nVv.jpg
http://pbs.twimg.com/ext_tw_video_thumb/1390740768780922883/pu/img/NCe0pcwOm14bKbUQ.jpg
http://pbs.twimg.com/media/E0zmQkpWEAIp-4h.jpg
http://pbs.twimg.com/media/E0zlyJFX0AEolsy.jpg
http://pbs.twimg.com/ext_tw_video_thumb/1390541364006461443/pu/img/A3NoE8X6G11Xa9n8.jpg
http://pbs.twimg.com/ext_tw_video_thumb/1390541364006461443/pu/img/A3NoE8X6G11Xa9n8.jpg
---------------------------

Autor do tweet:  ivetesangalo
Imagens postadas: 
---------------------------

Autor do tweet:  Tatawerneck
Imagens postadas: 
---------------------------



### Streaming

In [6]:
class MinhaStreamListener(tw.StreamListener):
    def on_status(self, status):
        print(status.user.screen_name)
        print(status.text)
        print('----------------------\n')
        

minhaStream = tw.Stream(auth=auth, listener=MinhaStreamListener())
minhaStream.filter(track='famosos')

### Azure

In [7]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

credenciais = CognitiveServicesCredentials(credentials['azure'])
client = ComputerVisionClient('https://westcentralus.api.cognitive.microsoft.com', credenciais)
client.api_version

### Análise de imagens

In [8]:
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes

url = 'http://pbs.twimg.com/media/ECx6hK-WwAAPzeE.jpg'
analize_de_imagem = client.analyze_image(url, VisualFeatureTypes.tags)
for tag in analize_de_imagem.tags:
    print(tag)

### Análise de celebridades

In [9]:
analise_celebridades = client.analyze_image_by_domain('celebrities', url, 'en')

for celebridade in analise_celebridades.result['celebrities']:
    print(celebridade['name'])
    print(celebridade['confidence'])

In [10]:
analise_celebridades.result['celebrities']

### Descrição da imagem

In [11]:
descricao = client.describe_image(url, 3, 'en')
for caption in descricao.captions:
    print(caption.text)
    print(caption.confidence)

In [12]:
descricao.captions[0].text

### Streaming + Vision API

In [14]:
famosos_id = []

for famoso in famosos:
    famoso_id = api.get_user(famoso).id_str
    famosos_id.append(famoso_id)

In [13]:
class MinhaStreamListener(tw.StreamListener):
    def on_status(self, status):
        print('Usuário:', status.user.screen_name)
        print('Texto:', status.text)
    
        if 'media' in status.entities:
            url = status.entities['media'][0]['media_url']
            print('URL: ', url)

            analise_celebridades = client.analyze_image_by_domain('celebrities', url, 'en')
            lista_celebridades = [celebridade['name'] for celebridade in analise_celebridades.result['celebrities']]
            print(lista_celebridades)

            descricao = client.describe_image(url, 1, 'en')
            texto_descricao = descricao.captions[0].text
            print('Descricao: ', texto_descricao)
            
            analize_de_imagem = client.analyze_image(url,VisualFeatureTypes.tags)
            lista_tags = [tag.name for tag in analize_de_imagem.tags]
            print('Tag: ', lista_tags)
      
            resultados = {
              'usuario': status.user.screen_name,
              'texto': status.text,
              'url': url,
              'celebridades': lista_celebridades,
              'descricao': texto_descricao,
              'tags': lista_tags
            }
        
        with open('tweets.txt', 'a') as arquivo:
            arquivo.write(json.dumps(resultados))
            arquivo.write('\n')

        print('-----')
        print('\n')
        
minhaStream = None
minhaStream = tw.Stream(auth = auth, listener=MinhaStreamListener())
minhaStream.filter(follow=famosos_id)

### Avaliando Resultados

In [15]:
lista_arquivo = []

with open('tweets.txt', 'r') as arquivo:
    for linha in arquivo:
        lista_arquivo.append(json.loads(linha))
        
imagens = pd.DataFrame(lista_arquivo)
imagens.head()

In [16]:
pesquisa = 'concert'
achados = [(pesquisa in lista) for lista in imagens.tags]
imagens[achados]

In [17]:
imagens[achados].descricao.iloc[0]

In [18]:
imagens[achados].tags.iloc[0]

In [19]:
imagens[achados].url.iloc[0]